Library installation and import

In [ ]:
%pip install requests
%pip install fastai
%pip install os-sys

#@title Install the required libs
%pip install -U -qq git+https://github.com/huggingface/diffusers.git
%pip install -qq accelerate transformers ftfy

#@title [Optional] Install xformers for faster and memory efficient training
#@markdown Acknowledgement: The xformers wheel are taken from [TheLastBen/fast-stable-diffusion](https://github.com/TheLastBen/fast-stable-diffusion). Thanks a lot for building these wheels!

%pip install -U --pre triton

In [2]:
import argparse
import itertools
import math
import os
import random

import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
from torch.utils.data import Dataset

import PIL
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed
from diffusers import AutoencoderKL, DDPMScheduler, PNDMScheduler, StableDiffusionPipeline, UNet2DConditionModel
from diffusers.optimization import get_scheduler
from diffusers.pipelines.stable_diffusion import StableDiffusionSafetyChecker
from PIL import Image
from torchvision import transforms
from tqdm.auto import tqdm
from transformers import CLIPFeatureExtractor, CLIPTextModel, CLIPTokenizer

from subprocess import getoutput
from IPython.display import HTML
from IPython.display import clear_output
import time

import requests
import matplotlib.pyplot as plt
from glob import glob

import shutil

In [1]:
#@title [Optional] Login to the Hugging Face Hub
#@markdown Add a token with the "Write Access" role to be able to add your trained concept to the [Library of Concepts](https://huggingface.co/sd-concepts-library)
from huggingface_hub import notebook_login

notebook_login()

Token has not been saved to git credential helper.


In [4]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
%%time
s = getoutput('nvidia-smi')
if 'T4' in s:
  gpu = 'T4'
elif 'P100' in s:
  gpu = 'P100'
elif 'V100' in s:
  gpu = 'V100'
elif 'A100' in s:
  gpu = 'A100'

while True:
    try: 
        gpu=='T4'or gpu=='P100'or gpu=='V100'or gpu=='A100'
        break
    except:
        pass
    print('[1;31mit seems that your GPU is not supported at the moment')
    time.sleep(5)

if (gpu=='T4'):
  %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/T4/xformers-0.0.13.dev0-py3-none-any.whl
  
elif (gpu=='P100'):
  %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/P100/xformers-0.0.13.dev0-py3-none-any.whl

elif (gpu=='V100'):
  %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/V100/xformers-0.0.13.dev0-py3-none-any.whl

elif (gpu=='A100'):
  %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/A100/xformers-0.0.13.dev0-py3-none-any.whl

Dataset download

In [5]:
def download_specific_characters(character_ids):
    """Download specific character datasets by their ID numbers"""

    base_url = "https://huggingface.co/datasets/BangumiBase/onepiece/resolve/main"

    for char_id in character_ids:
        url = f"{base_url}/{char_id}/dataset.zip"
        filename = f"character_{char_id}.zip"

        print(f"Downloading character {char_id}...")
        response = requests.get(url, stream=True)

        if response.status_code == 200:
            with open(filename, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Downloaded {filename}")
        else:
            print(f"Failed to download character {char_id}")


In [ ]:
download_specific_characters([2,7,8,12,14,63,29,33,34,35,55,56,60,65,69,125,126,149])

In [ ]:
%%time
# Create character directories:
!mkdir -p chopper
!mkdir -p franky
!mkdir -p god_usopp
!mkdir -p luffy
!mkdir -p nami
!mkdir -p robin
!mkdir -p sanji
!mkdir -p sabo
!mkdir -p shanks
!mkdir -p zoro

# Unzip commands:
!unzip character_2.zip -d shanks
!unzip character_7.zip -d luffy
!unzip character_8.zip -d luffy
!unzip character_12.zip -d god_usopp
!unzip character_14.zip -d god_usopp
!unzip character_29.zip -d zoro
!unzip character_33.zip -d sanji
!unzip character_34.zip -d sanji
!unzip character_35.zip -d sanji
!unzip character_55.zip -d chopper
!unzip character_56.zip -d chopper
!unzip character_60.zip -d franky
!unzip character_63.zip -d luffy
!unzip character_65.zip -d robin
!unzip character_69.zip -d robin
!unzip character_125.zip -d nami
!unzip character_126.zip -d nami
!unzip character_149.zip -d sabo

In [ ]:
def quick_preview(character_folder, num_images=9):
    """Quick 3x3 grid preview"""
    import glob
    
    image_files = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.webp']:
        image_files.extend(glob.glob(os.path.join(character_folder, ext)))
    
    if not image_files:
        print(f"No images found in {character_folder}")
        return
    
    fig, axes = plt.subplots(3, 3, figsize=(10, 10))
    fig.suptitle(f'{character_folder.title()} - {len(image_files)} total images')
    
    for i in range(9):
        row, col = i // 3, i % 3
        if i < len(image_files):
            img = Image.open(image_files[i])
            axes[row, col].imshow(img)
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()
    print(f"Total images in {character_folder}: {len(image_files)}")

# Quick preview all characters
for character in ['shanks', 'luffy', 'zoro', 'sanji','franky','nami','robin','sabo','god_usopp']:
    quick_preview(character)

In [5]:
# --- Helpers used in show_image ---
def hasattrs(obj, attrs): return all(hasattr(obj, attr) for attr in attrs)
def array(x): return np.array(x)
def ifnone(a, b): return b if a is None else a
def _fig_bounds(x): return min(10, max(3, x // 20))

# --- show_image function ---
def show_image(im, ax=None, figsize=None, title=None, ctx=None, **kwargs):
    if hasattrs(im, ('data','cpu','permute')):
        im = im.data.cpu()
        if im.shape[0]<5: im=im.permute(1,2,0)
    elif not isinstance(im, np.ndarray): im = array(im)
    if im.shape[-1] == 1: im = im[..., 0]

    ax = ifnone(ax, ctx)
    if figsize is None: figsize = (_fig_bounds(im.shape[0]), _fig_bounds(im.shape[1]))
    if ax is None: _, ax = plt.subplots(figsize=figsize)
    ax.imshow(im, **kwargs)
    if title is not None: ax.set_title(title)
    ax.axis('off')
    return ax

# --- Image folders list ---
folders = [
    "./chopper", "./robin", "./franky", "./sabo", "./god_usopp",
    "./sanji", "./luffy", "./shanks", "./zoro"
]

# --- Collect image paths (e.g. only first image from each folder) ---
image_paths = []
for folder in folders:
    imgs = glob(os.path.join(folder, "*.jpg")) + glob(os.path.join(folder, "*.png"))
    if imgs:
        image_paths.append(imgs[0])  # pick the first image from each folder

# --- Display the images ---
n = len(image_paths)
fig, axs = plt.subplots(1, n, figsize=(n * 4, 4))

# Ensure axs is iterable even if n==1
if n == 1:
    axs = [axs]

for ax, path in zip(axs, image_paths):
    im = Image.open(path)
    title = os.path.basename(os.path.dirname(path))  # folder name as title
    show_image(im, ax=ax, title=title)

plt.tight_layout()
plt.show()


# Textual Inversion

Settings for teaching your new concept

In [6]:
#@markdown `pretrained_model_name_or_path` which Stable Diffusion checkpoint you want to use
#@markdown `pretrained_model_name_or_path` which Stable Diffusion checkpoint you want to use
pretrained_model_name_or_path = "stabilityai/stable-diffusion-2" #@param ["stabilityai/stable-diffusion-2", "stabilityai/stable-diffusion-2-base", "CompVis/stable-diffusion-v1-4", "runwayml/stable-diffusion-v1-5"] {allow-input: true}

## Load images from local folder

In [ ]:
# Method 1: Specify multiple image paths directly
image_folders = [
   "./chopper",
   "./robin", 
   "./franky",
   "./sabo",
   "./god_usopp",
   "./sanji",
   "./luffy",
   "./shanks",
   "./zoro"
]

# Create a combined directory for all images
combined_save_path = "./combined_concept_images"
os.makedirs(combined_save_path, exist_ok=True)

# Copy all images from multiple folders to the combined directory
image_count = 0
for folder_path in image_folders:
   if os.path.exists(folder_path):
       print(f"Processing folder: {folder_path}")
       for file_name in os.listdir(folder_path):
           file_path = os.path.join(folder_path, file_name)
           try:
               # Test if it's a valid image
               with Image.open(file_path) as img:
                   # Copy with a unique name to avoid conflicts
                   new_name = f"image_{image_count}_{file_name}"
                   new_path = os.path.join(combined_save_path, new_name)
                   shutil.copy2(file_path, new_path)
                   image_count += 1
                   print(f"Copied: {file_name}")
           except Exception as e:
               print(f"Skipped {file_name}: Not a valid image")
   else:
       print(f"Folder does not exist: {folder_path}")

print(f"Total images collected: {image_count}")
save_path = combined_save_path

### Setup and check the images you have just added

In [11]:
images = []
for file_path in os.listdir(save_path):
  try:
      image_path = os.path.join(save_path, file_path)
      images.append(Image.open(image_path).resize((512, 512)))
  except:
    print(f"{image_path} is not a valid image, please make sure to remove this file from the directory otherwise the training could fail.")
image_grid(images, 1, len(images))

: 

In [8]:
# Just validate files without loading them into memory
valid_count = 0
invalid_files = []

print("Validating training images...")
for file_path in os.listdir(save_path):
    try:
        image_path = os.path.join(save_path, file_path)
        # Just open and immediately close - don't store in memory
        with Image.open(image_path) as img:
            if img.size[0] > 0 and img.size[1] > 0:  # Basic validation
                valid_count += 1
                if valid_count <= 5:  # Only print first 5
                    print(f"✓ {file_path}: {img.size}")
    except Exception as e:
        invalid_files.append(file_path)
        print(f"✗ Invalid: {file_path}")

print(f"\nValidation complete: {valid_count} valid images")
if invalid_files:
    print(f"Invalid files to remove: {invalid_files}")

# Set empty list for compatibility with later code
images = []

Validating training images...


✓ image_0_10026.png: (423, 538)
✓ image_10000_562.png: (638, 894)
✓ image_10001_5625.png: (511, 480)
✓ image_10002_5627.png: (493, 479)
✓ image_10003_563.png: (1388, 1080)

Validation complete: 12743 valid images


In [9]:
#@title Settings for your newly created concept
#@markdown `what_to_teach`: what is it that you are teaching? `object` enables you to teach the model a new object to be used, `style` allows you to teach the model a new style one can use.
what_to_teach = "style" #@param ["object", "style"]
#@markdown `placeholder_token` is the token you are going to use to represent your new concept (so when you prompt the model, you will say "A `<my-placeholder-token>` in an amusement park"). We use angle brackets to differentiate a token from other words/tokens, to avoid collision.
placeholder_token = "<one-piece-styling>" #@param {type:"string"}
#@markdown `initializer_token` is a word that can summarise what your new concept is, to be used as a starting point
initializer_token = "one-piece-style" #@param {type:"string"}

### Teach the model a new concept (fine-tuning with textual inversion)

Execute this this sequence of cells to run the training process. The whole process may take from 1-4 hours. 

#### Create dataset

In [10]:
#@title Setup the prompt templates for training 
imagenet_templates_small = [
    "a photo of a {}",
    "a rendering of a {}",
    "a cropped photo of the {}",
    "the photo of a {}",
    "a photo of a clean {}",
    "a photo of a dirty {}",
    "a dark photo of the {}",
    "a photo of my {}",
    "a photo of the cool {}",
    "a close-up photo of a {}",
    "a bright photo of the {}",
    "a cropped photo of a {}",
    "a photo of the {}",
    "a good photo of the {}",
    "a photo of one {}",
    "a close-up photo of the {}",
    "a rendition of the {}",
    "a photo of the clean {}",
    "a rendition of a {}",
    "a photo of a nice {}",
    "a good photo of a {}",
    "a photo of the nice {}",
    "a photo of the small {}",
    "a photo of the weird {}",
    "a photo of the large {}",
    "a photo of a cool {}",
    "a photo of a small {}",
]

imagenet_style_templates_small = [
    "a painting in the style of {}",
    "a rendering in the style of {}",
    "a cropped painting in the style of {}",
    "the painting in the style of {}",
    "a clean painting in the style of {}",
    "a dirty painting in the style of {}",
    "a dark painting in the style of {}",
    "a picture in the style of {}",
    "a cool painting in the style of {}",
    "a close-up painting in the style of {}",
    "a bright painting in the style of {}",
    "a cropped painting in the style of {}",
    "a good painting in the style of {}",
    "a close-up painting in the style of {}",
    "a rendition in the style of {}",
    "a nice painting in the style of {}",
    "a small painting in the style of {}",
    "a weird painting in the style of {}",
    "a large painting in the style of {}",
]

In [ ]:
#@title Setup the dataset
class TextualInversionDataset(Dataset):
    def __init__(
        self,
        data_root,
        tokenizer,
        learnable_property="object",  # [object, style]
        size=512,
        repeats=100,
        interpolation="bicubic",
        flip_p=0.5,
        set="train",
        placeholder_token="*",
        center_crop=False,
    ):

        self.data_root = data_root
        self.tokenizer = tokenizer
        self.learnable_property = learnable_property
        self.size = size
        self.placeholder_token = placeholder_token
        self.center_crop = center_crop
        self.flip_p = flip_p

        self.image_paths = [os.path.join(self.data_root, file_path) for file_path in os.listdir(self.data_root)]

        self.num_images = len(self.image_paths)
        self._length = self.num_images

        if set == "train":
            self._length = self.num_images * repeats

        self.interpolation = {
            "linear": PIL.Image.LINEAR,
            "bilinear": PIL.Image.BILINEAR,
            "bicubic": PIL.Image.BICUBIC,
            "lanczos": PIL.Image.LANCZOS,
        }[interpolation]

        self.templates = imagenet_style_templates_small if learnable_property == "style" else imagenet_templates_small
        self.flip_transform = transforms.RandomHorizontalFlip(p=self.flip_p)

    def __len__(self):
        return self._length

    def __getitem__(self, i):
        example = {}
        image = Image.open(self.image_paths[i % self.num_images])

        if not image.mode == "RGB":
            image = image.convert("RGB")

        placeholder_string = self.placeholder_token
        text = random.choice(self.templates).format(placeholder_string)

        example["input_ids"] = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.tokenizer.model_max_length,
            return_tensors="pt",
        ).input_ids[0]

        # default to score-sde preprocessing
        img = np.array(image).astype(np.uint8)

        if self.center_crop:
            crop = min(img.shape[0], img.shape[1])
            h, w, = (
                img.shape[0],
                img.shape[1],
            )
            img = img[(h - crop) // 2 : (h + crop) // 2, (w - crop) // 2 : (w + crop) // 2]

        image = Image.fromarray(img)
        image = image.resize((self.size, self.size), resample=self.interpolation)

        image = self.flip_transform(image)
        image = np.array(image).astype(np.uint8)
        image = (image / 127.5 - 1.0).astype(np.float32)

        example["pixel_values"] = torch.from_numpy(image).permute(2, 0, 1)
        return example

#### Setting up the model

Below we set up our tokenizer

In [ ]:
tokenizer = CLIPTokenizer(
    pretrained_model_or_path,
    subfolder = "tokenizer",
)

num_added_tokens = tokenizer.add_tokens(placeholder_token)
if num_added_tokens == 0:
    raise ValueError(
        f"The tokenizer already contains the token {palceholder_token}. Please pass a different"
        "`placeholder_token` that is not already in the tokenizer"
    )

#### Load the Stable Diffusion model

Load models and create wrapper for stable diffusion

```pipeline = StableDiffusionPipeline.from_pretrained(pretrained_model_name_or_path)```


```del pipeline```

text_encoder = CLIPTextModel.from_pretrained(pretrained_model_or_path,subfolder="text_encoder")

In [ ]:
vae = AutoencoderKL.from_pretrained(pretrained_model_or_path,subfolder="vae")

In [ ]:
unet = UNet2DConditionModel.from_pretrained(pretrained_model_or_path,subfolder="unet")

We can now resize our token embeddings to include our newly added placeholder_token. We have added the placeholder_token in the tokenizer so we resize the token embeddings here, this will a new embedding vector in the token embeddings for our placeholder_token

In [ ]:
text_encoder.resize_token_embeddings(len(tokenizer))

Initialise the newly added placeholder token with the embeddings of the initializer token instead of random initialization.

In [ ]:
token_embeds = text_encoder.get_input_embeddings().weight.data
token_embeds[placeholder_token_id] = token_embeddings[initializer_token_id]

In [ ]:
def freeze_params(params):
    for params in params:
        param.requires_grad = False

freeze_params(vae.parameters())
freeze_params(unet.parameters())

params_to_freeze = itertools.chain(
    text_encoder.text_model.encoder.parameters(),
    text_encoder.text_model.final_layer_norm.parameters(),
    text_encoder.text_model.embeddings.position_embedding.parameters(),
)
freeze_params(params_to_freeze)

Create our training data

In [ ]:
train_dataset

In [ ]:
from diffusers import AutoencoderKL, DDPMScheduler, PNDMScheduler, StableDiffusionPipeline, UNet2DConditionModel

Load images from local folder or google drive